In [1]:
import pandas as pd
import numpy as np

In [166]:
# df_news_tweets = pd.read_pickle('en-emd-paraphrase-distilroberta-base-v2.pkl')
# df_news_tweets = pd.read_pickle('en-emd-paraphrase-mpnet-base-v2.pkl')
# df_news_tweets = pd.read_pickle('en-emd-paraphrase-MiniLM-L6-v2.pkl')
# df_news_tweets = pd.read_pickle('en-emd-nli-mpnet-base-v2.pkl')
# df_news_tweets = pd.read_pickle('en-emd-nli-roberta-base-v2.pkl')
# df_news_tweets = pd.read_pickle('en-emd-digitalepidemiologylab-covid-twitter-bert-v2.pkl')
# df_news_tweets = pd.read_pickle('en-emd-cardiffnlp-twitter-roberta-base.pkl')
df_news_tweets = pd.read_pickle('en-emd-facebook-laser.pkl')

In [125]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [10]:
from sentence_transformers import util
from sklearn import metrics
from sklearn.metrics import pairwise_distances

### Suppose tweets division is the ground truth, Let's calculate the metrices and compare

In [131]:
# communities are news ids
tweet_id = []
label = []

news_id = set(df_news_tweets['news_id'])
i = 0
for idn in news_id:
    news = df_news_tweets[df_news_tweets["news_id"] == idn]
    if (len(news) >= 5):
        for index, row in news.iterrows():
            tweet_id.append(row['tweet_id'])
            label.append(i)
        
        i += 1
        
tweet_label = {
    'tweet_id':tweet_id,
    'news_label': label
}
df_news_labels = pd.DataFrame(tweet_label, columns = ['tweet_id', 'news_label'])

In [132]:
df_news_com = pd.merge(df_news_tweets, df_news_labels, on = 'tweet_id', how = 'inner')

In [133]:
print(len(set(df_news_tweets['news_id'].tolist()))) # Number of news (clusters)

961


In [134]:
X_news = df_news_com['tweet_embeddings'].tolist()
labels_news = df_news_com['news_label'].tolist()
sil_news = metrics.silhouette_score(X_news, labels_news, metric='cosine')
sil_news

0.3186821315736031

### Cosine similarity matrix

In [167]:
cos_mat = util.cos_sim(df_news_tweets['tweet_embeddings'].tolist(),df_news_tweets['tweet_embeddings'].tolist())

In [168]:
cos_copy = cos_mat.detach().cpu().numpy()

In [169]:
def cal_sil_coef(df_coms, df_news_tweets):
    '''
    calculate Silhouette coefficient 
    
    input:
    df_coms: a dictionary of dataframes, each dataframe is one cluster subset of dataframe df_news_tweets
    df_news_tweets: input dataframe that contains all tweets and other metadata information
    
    return:
    df_tweets_labels: df_news_tweets added cluster labels
    sil: Silhouette coefficient of the clustering result
    '''
        
    tweet_id = []
    label = []
    for i in range(len(df_coms)):
        for index, row in df_coms[i].iterrows():
            tweet_id.append(row['tweet_id'])
            label.append(i)

    tweet_label = {
        'tweet_id':tweet_id,
        'com_label': label
    }
    df_labels = pd.DataFrame(tweet_label, columns = ['tweet_id', 'com_label'])
    df_tweets_labels = pd.merge(df_news_tweets, df_labels, on = 'tweet_id', how='inner')
    
    X = df_tweets_labels['tweet_embeddings'].tolist()
    labels = df_tweets_labels['com_label'].tolist()
    sil = metrics.silhouette_score(X, labels, metric='cosine')
    
    return df_tweets_labels, sil

### Agglomerative clustering

In [170]:
from sklearn.cluster import AgglomerativeClustering

In [171]:
corpus = df_news_tweets['clean_text'].tolist()

In [172]:
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=0.15, affinity='precomputed', linkage='average')

In [173]:
clustering_model.fit(1-cos_copy)
cluster_assignment = clustering_model.labels_

In [174]:
clustered_sentences = {}
clustered_ids = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []
        clustered_ids[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])
    clustered_ids[cluster_id].append(sentence_id)

In [175]:
communities = sorted(clustered_ids.items(), key= lambda x: len(x[1]), reverse=True) 

In [176]:
len(communities)

10474

In [177]:
dict_com = {}
k = 0
for com in communities:
    if (len(com[1])>4):
        dict_com[k] = df_news_tweets.loc[df_news_tweets.index[com[1]]]
        k += 1

In [178]:
len(dict_com)

833

In [183]:
i = 2
print(len(dict_com[i]))
for index, row in dict_com[i].iterrows():
        print(row['clean_text'])
        print(row['tweet_id'])

105
* what are your thoughts on this story? * ️ The Japanese professor of physiology or medicine, Professor Dr Tasuku Honjo, caused a sensation today in the media by saying that the corona virus is not natural. 
1292405626342199296
Thread by A Japanese medicine professor, Professor Dr Tasuku Honjo, caused a sensation today in the media by saying that COVID 19 isnatural. If it is natural, it will not have affected the whole world like that. Because, depending on the nature,…
1291079009359613953
* SHOCKING * ️ The Japanese professor of physiology or medicine, Professor Dr Tasuku Honjo, caused a sensation today in the media by saying that the corona virus is not natural. if it is natural, it will not have affected the whole world!
1291058529789386753
Thread by A Japanese medicine professor, Professor Dr Tasuku Honjo, caused a sensation today in the media by saying that COVID 19 isnatural. If it is natural, it will not have affected the whole world like that. Because, depending on the natu

In [180]:
df_cluster_labels, agglo_sil = cal_sil_coef(dict_com, df_news_tweets) # Silhouette coefficient for agglomerative clustering

In [181]:
print(agglo_sil)

0.7433601452585903


In [110]:
df_cluster_labels.to_pickle('Agglomerative_CosTh85.pkl')

### Leiden community detection

In [184]:
import networkx as nx
import leidenalg as la
import igraph as ig

In [185]:
cos_copy[cos_copy < 0.85] = 0

In [186]:
G = nx.from_numpy_array(cos_copy)
g = ig.Graph.from_networkx(G)
communities = la.find_partition(g, la.ModularityVertexPartition, n_iterations=-1)
optimizer = la.Optimiser()        
diff = 1
while diff > 0:
    diff = optimizer.optimise_partition(communities,n_iterations=-1)

In [187]:
len(communities)

9289

In [188]:
dict_com = {}
k = 0
for i in range(len(communities)):
    if (len(communities[i])>4):
        dict_com[k] = df_news_tweets.loc[df_news_tweets.index[communities[i]]]
        k += 1

In [189]:
print(len(dict_com))

654


In [197]:
i = 2
print(len(dict_com[i]))
for index, row in dict_com[i].iterrows():
        print(row['clean_text'])
        print(row['tweet_id'])

238
Please dont drink Dettol given by anybody to stop coronavirus Corona Virus Updates COVIDIOTS Covid_19india Coronaindia Corona Updates In India coronavirusinpakistan Coronavirus InSA 
1242101504204390405
Please dont drink Dettol given by anybody to stop coronavirus Corona Virus Updates COVIDIOTS Covid_19india Coronaindia Corona Updates In India coronavirusinpakistan Coronavirus InSA 
1242101138423349251
Large-scale coronavirus vaccine trial begins in South Africa Corona Virus Corona Virus Outbreak COVID2019 covid19 medtwitter virus Corona Virus Challenge Corona pandemic batflu Wuhan Quarantine socialdistancing
1276681781803171842
Is This Train Car Carrying ‘COVID-19’? True or False. covid19traintanker covidupdateinus covid19us COVIDー19 COVID19 Covid_19ireland coronavirus Corona Virus Updates Corona Katie covidcases covid19traintankercar
1239064794717491201
Mayo Clinic creates drive-through specimen collection for coronavirustesting coronavirus Coronavirus Outbreak COVID19 COVIDー19 C

In [198]:
news = set(dict_com[i]['news_summary'])
news_id = set(dict_com[i]['news_id'])
print(news)
print(len(news))

{'Coronavirus is forcing fans of Bitcoin to realize it’s not a “safe haven” after all', 'Why simply waiting for herd immunity to covid-19 isn’t an option', 'How the space industry has adjusted to life under coronavirus', 'A trial is under way of the first new antibody medicine developed to treat covid-19', 'A new CDC bot will tell you what to do if you have coronavirus symptoms', 'Nearly 40% of Icelanders are using a covid app—and it hasn’t helped much', 'Antigen testing could be a faster, cheaper way to diagnose covid-19', 'Facebook has released a map of coronavirus symptoms crowdsourced from its users', 'AI is helping triage coronavirus patients. The tools may be here to stay.', 'Apple and Google’s covid-tracing tech has been released to 23 countries', 'We need mass surveillance to fight covid-19—but it doesn’t have to be creepy', 'How does the coronavirus work?', ' The spread of the new coronavirus was planned by the Rockefellers years ago.', 'The coronavirus outbreak may exacerbate

In [192]:
df_cluster_labels, leiden_sil = cal_sil_coef(dict_com, df_news_tweets) # Silhouette coefficient for Leiden community detection

In [193]:
print(leiden_sil)

0.6040375506596777


In [123]:
df_cluster_labels.to_pickle('Leiden_CosTh85.pkl')